<a href="https://colab.research.google.com/github/kanishka55/import_data/blob/main/second_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
import string

import pandas as pd
import matplotlib.pyplot as plt

from time import perf_counter 

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [3]:
!git clone https://github.com/filipefilardi/text-classification.git

Cloning into 'text-classification'...
remote: Enumerating objects: 18983, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 18983 (delta 28), reused 51 (delta 22), pack-reused 18918
Receiving objects: 100% (18983/18983), 113.87 MiB | 21.46 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [4]:
df_full = pd.read_csv('../content/text-classification/metrics/models_evaluation.csv', sep=';')

In [5]:
df_full

,model_name,fold,time_train,time_application,accuracy
0,Gaussian Naive Bayes (var_smoothing=1e-09),1,11.648,15.963,0.660392
1,"MLP (layers=10, lr=0.001, relu)",1,290.855,0.278,0.788441
2,"MLP (layers=100, lr=0.001, relu)",1,881.979,0.360,0.823171
3,"MLP (layers=132, lr=0.001, relu)",1,1109.083,0.425,0.814952
4,"MLP (layers=10, lr=0.01, relu)",1,103.463,0.217,0.786585
...,...,...,...,...,...
160,"Random Forest (n_tree=1000, max_feat=log2)",5,296.300,5.089,0.804735
161,KNN (n_neighbors=5),5,22.794,1678.101,0.314711
162,KNN (n_neighbors=27),5,49.617,2872.623,0.141261
163,KNN (n_neighbors=81),5,73.122,4203.200,0.728119


In [6]:
raw_full = pd.read_csv('/content/text-classification/data/processed/20newsgroup_raw.csv', sep=';')

In [7]:
raw_full

,target,text
0,alt.atheism,From: mathew <mathew@mantis.co.uk>\nSubject: A...
1,alt.atheism,From: mathew <mathew@mantis.co.uk>\nSubject: A...
2,alt.atheism,From: I3150101@dbstu1.rz.tu-bs.de (Benedikt Ro...
3,alt.atheism,From: mathew <mathew@mantis.co.uk>\nSubject: R...
4,alt.atheism,From: strom@Watson.Ibm.Com (Rob Strom)\nSubjec...
...,...,...
18823,talk.religion.misc,From: sbuckley@fraser.sfu.ca (Stephen Buckley)...
18824,talk.religion.misc,From: bakerj@gtephx.UUCP (Jon Baker)\nSubject:...
18825,talk.religion.misc,From: pharvey@quack.kfu.com (Paul Harvey)\nSub...
18826,talk.religion.misc,From: <KEVXU@CUNYVM.BITNET>\nSubject: Re: Info...


In [8]:
raw_full.head()

,target,text
0,alt.atheism,From: mathew <mathew@mantis.co.uk>\nSubject: A...
1,alt.atheism,From: mathew <mathew@mantis.co.uk>\nSubject: A...
2,alt.atheism,From: I3150101@dbstu1.rz.tu-bs.de (Benedikt Ro...
3,alt.atheism,From: mathew <mathew@mantis.co.uk>\nSubject: R...
4,alt.atheism,From: strom@Watson.Ibm.Com (Rob Strom)\nSubjec...


In [9]:
print(raw_full.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18828 entries, 0 to 18827
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   target  18828 non-null  object
 1   text    18828 non-null  object
dtypes: object(2)
memory usage: 294.3+ KB
None


In [10]:
print(raw_full.head(5))

        target                                               text
0  alt.atheism  From: mathew <mathew@mantis.co.uk>\nSubject: A...
1  alt.atheism  From: mathew <mathew@mantis.co.uk>\nSubject: A...
2  alt.atheism  From: I3150101@dbstu1.rz.tu-bs.de (Benedikt Ro...
3  alt.atheism  From: mathew <mathew@mantis.co.uk>\nSubject: R...
4  alt.atheism  From: strom@Watson.Ibm.Com (Rob Strom)\nSubjec...


In [11]:
print(raw_full)

                   target                                               text
0             alt.atheism  From: mathew <mathew@mantis.co.uk>\nSubject: A...
1             alt.atheism  From: mathew <mathew@mantis.co.uk>\nSubject: A...
2             alt.atheism  From: I3150101@dbstu1.rz.tu-bs.de (Benedikt Ro...
3             alt.atheism  From: mathew <mathew@mantis.co.uk>\nSubject: R...
4             alt.atheism  From: strom@Watson.Ibm.Com (Rob Strom)\nSubjec...
...                   ...                                                ...
18823  talk.religion.misc  From: sbuckley@fraser.sfu.ca (Stephen Buckley)...
18824  talk.religion.misc  From: bakerj@gtephx.UUCP (Jon Baker)\nSubject:...
18825  talk.religion.misc  From: pharvey@quack.kfu.com (Paul Harvey)\nSub...
18826  talk.religion.misc  From: <KEVXU@CUNYVM.BITNET>\nSubject: Re: Info...
18827  talk.religion.misc  From: pharvey@quack.kfu.com (Paul Harvey)\nSub...

[18828 rows x 2 columns]


In [12]:
dataset = pd.read_csv('/content/text-classification/data/processed/20newsgroup_preprocessed.csv',sep=';')

In [13]:
dataset

,target,text,text_cleaned
0,alt.atheism,From: mathew <mathew@mantis.co.uk>\nSubject: A...,atheist resources addresses atheist organizati...
1,alt.atheism,From: mathew <mathew@mantis.co.uk>\nSubject: A...,begin pgp signed message introduction atheism ...
2,alt.atheism,From: I3150101@dbstu1.rz.tu-bs.de (Benedikt Ro...,article charley wingate writes well john quite...
3,alt.atheism,From: mathew <mathew@mantis.co.uk>\nSubject: R...,kings become philosophers philosophers become ...
4,alt.atheism,From: strom@Watson.Ibm.Com (Rob Strom)\nSubjec...,article bob mcgwier writes however hate econom...
...,...,...,...
18823,talk.religion.misc,From: sbuckley@fraser.sfu.ca (Stephen Buckley)...,paul boxrud writes wasnt sure right newsgroup ...
18824,talk.religion.misc,From: bakerj@gtephx.UUCP (Jon Baker)\nSubject:...,article joakim ruud writes article mark wilson...
18825,talk.religion.misc,From: pharvey@quack.kfu.com (Paul Harvey)\nSub...,article bill jefferys writes would like unders...
18826,talk.religion.misc,From: <KEVXU@CUNYVM.BITNET>\nSubject: Re: Info...,article gerry palo danger anticult groups expo...


In [17]:
df_full = pd.read_csv('/content/text-classification/data/processed/20newsgroup_raw.csv', sep=';')

In [18]:
%%time

def clean_header(text):
    text = re.sub(r'(From:\s+[^\n]+\n)', '', text)
    text = re.sub(r'(Subject:[^\n]+\n)', '', text)
    text = re.sub(r'(([\sA-Za-z0-9\-]+)?[A|a]rchive-name:[^\n]+\n)', '', text)
    text = re.sub(r'(Last-modified:[^\n]+\n)', '', text)
    text = re.sub(r'(Version:[^\n]+\n)', '', text)

    return text


df_full['text_cleaned'] = df_full['text'].apply(clean_header)

CPU times: user 30.9 s, sys: 0 ns, total: 30.9 s
Wall time: 31 s


In [20]:
%%time

def clean_text(text):        
    text = text.lower()
    text = text.strip()
  
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    text = re.sub(r'(\d+)', ' ', text)
    text = re.sub(r'(\s+)', ' ', text)
    
    return text

df_full['text_cleaned'] = df_full['text_cleaned'].apply(clean_text)

CPU times: user 3.5 s, sys: 0 ns, total: 3.5 s
Wall time: 3.5 s
